### Códigos de Agentes Regulados
Notebook para extração e implementação no Big Query da tabela SIMP

In [18]:
from io import BytesIO
from google.cloud import bigquery
import zipfile
import os
import requests
import zipfile
import pandas as pd

In [19]:
BASE_URL = "https://csa.anp.gov.br/downloads/manuais-isimp/Geracao-de-tabelas-ISIMP.zip"

In [20]:
response = requests.get(BASE_URL, verify=False)
if response.status_code == 200:
    pass
else:
    raise Exception(f"Failed to download file: {response.status_code}")

with zipfile.ZipFile(BytesIO(response.content)) as z:
    with z.open("T001-Codigos_agentes_regulados.xlsx") as excel_file:
        df = pd.read_excel(excel_file, header=1)
df = df.rename(columns={
    'Cod. Agente ANP': 'cod_agente_anp',
    'Raiz CNPJ': 'raiz_cnpj', 
    'Razão Social': 'razao_social',
    'CEP': 'cep',
    'Endereco': 'endereco',
    'Bairro': 'bairro',
    'Municipio': 'municipio',
    'Estado': 'estado',
    'Tipo do Agente': 'tipo_do_agente',
    'Data Inicio Validade': 'data_inicio_validade',
    'Data Final Validade': 'data_final_validade',
    'dataVersao': 'data_versao'
})
df['raiz_cnpj'] = df['raiz_cnpj'].astype(str).str.zfill(8)
df['data_upload'] = pd.to_datetime('today')
print(df['data_upload'].dtype)

/Users/dwbessa/projects/vibra/pipeline_biomass_calculation/.venv/lib/python3.13/site-packages/urllib3/connectionpool.py:1097: InsecureRequestWarning: Unverified HTTPS request is being made to host 'csa.anp.gov.br'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#tls-warnings
  warnings.warn(


datetime64[ns]


In [21]:
"""
Inserting data into BigQuery
"""
bq_client = bigquery.Client()
project_id = os.getenv("GOOGLE_CLOUD_PROJECT")
bq_dataset = "rw_ext_anp"
table_name = "dados_agentes"
table_id = f"{project_id}.{bq_dataset}.{table_name}"

job_config = bigquery.LoadJobConfig(
    write_disposition=bigquery.WriteDisposition.WRITE_TRUNCATE,
)

try:
    job = bq_client.load_table_from_dataframe(
        df, table_id, job_config=job_config
    )
    job.result()
    print(f"  Data for {table_id} inserted successfully.")
except Exception as e:
    print(f"  Error inserting data for {table_id}: {str(e)}")

print("Data insertion completed!")

  Data for labs-vibra-final.rw_ext_anp.dados_agentes inserted successfully.
Data insertion completed!
